# <p style="text-align: center;">FIT5196 Assignment 1 (Bonus)</p>

### Student Name : Robert
### Student ID       : 28243447

Last Updated: 2 September 2018

## 1. Extract The Required Qualifications From the Job Advertisements

The dataset use in this section is `28243447.dat`

The code below is taken from Task 1.

In [14]:
import re # library for regular expression

#for opening the file
with open('28243447.dat','r') as infile:
    text = infile.read()
    
# the regular expression
regex= r'(^ID:|\n(?:JOB)?[_ ]?[T|t](?:TL)?(?:ITLE)?(?:itle)?S?:|\n(?:JOB|(?:_?LOC(?:ATION)?S?))+:|\n(?:JOB[_ ]|job|DESC|RIPTION|_?desc|ription)+:|\n(?:JOB[_ ]|(?:RESP|resp)\w*)+:|\n(?:REQ\w{5}[S ]|(?:QUALIF|qualif)S?|(?:ICATION|ication)[S|s]?)+:|\n(?:JOB_?|SAL|sal|ARY|ary|(?:REMU|remu)\w*)+:|\n(?:JOB_?|PROC|proc|EDURE|S|edures)+:|\n(?:[S|s]\w{3}[T|t]|[_ ]?(?:DA|da)|(?:TE|te)|[_ ]|S)+:|\n(?:APPL\w{7}|[D|d](?:EAD|ead)_?(?:LINES?|line)?|_(?:DL|DEADL))+:|\n(?:[A|a]\w{3}[T|t][_ ]?|(?:COMP|comp)\w{3}[S|s]?_?|_[I|i]\w{2}[O|o])+:)'

# split the main regular expression into 11 parts of small regular expressions in order to identify captured label
regex_split = regex.split("|\\n")
for i in range(len(regex_split)):
    if i == 0: regex_split[i]=regex_split[i]+')'
    elif i==len(regex_split)-1: regex_split[i] = '(\\n'+regex_split[i]
    else: regex_split[i]='(\\n'+regex_split[i]+')'
        

# This column will be used in order to control the order of the output
columns = ['_id','title', 'location', 'job_descriptions','job_responsibilities', 'required_qualifications', 'salary','application_procedure','start_date', 'application_deadline', 'about_company']

# Split the raw text file by using 30 of '-' as a delimiter
rawtext = re.split(r'-{30}', text)

data = {}  
data['listing'] = []  

# Loop through all job vacancies
for index in range (0,500):
    # Find all the corresponding labels by using the regular expression
    purepattern=re.findall(regex,rawtext[index], re.M)
    concatpattern = "("+'|'.join(purepattern)+"|\n \n)"
    
    # Split the raw text by using the result of regular expression
    splitted_text = re.split(concatpattern, rawtext[index])
    
    # Variable to store the processed raw data
    listing_dict = dict.fromkeys(columns)
    
    # Loop through all the raw text to find the content of the corresponding label
    for i in range (len(regex_split)):
        for j in range (len(splitted_text)):
            if (re.match(regex_split[i],splitted_text[j])!=None):
                processing = splitted_text[j+1].replace("\t","")
                if listing_dict[columns[i]] == None:
                    listing_dict[columns[i]] = [processing]
                else:
                    listing_dict[columns[i]] += [processing]
                    
    # Loop through all the result in the dictionary in order to handle the children of job description, job_responsibilities and required qualification
    for key, value in listing_dict.items():
        if(key in ('job_descriptions','job_responsibilities', 'required_qualifications')):
            child_dict = {}
            if(value is not None):
                child_list = []
                for each in value:
                    # Use regular expression to find the children which start with '-' and end with ; or end of the text
                    # The meaning of the regular expression is lookbehind equal '- ' until meet ';' character
                    child_item = re.findall(r'((?<=- )[^;]*)',each, re.M)
                    if(child_item == []):
                        child_list.append(each)
                    else:
                        child_list.extend(child_item)
                # Give the label to the children
                if(key=="job_descriptions"): child_key = "description"
                elif(key=="job_responsibilities"): child_key="responsibility"
                else: child_key="qualification"
                # Store the result
                child_dict[child_key]=child_list
                listing_dict[key] = child_dict
    
    # Handle the empty value or missing label
    for key, value in listing_dict.items():
        if value is None:
            listing_dict[key] = 'N/A'
        elif key in ('job_descriptions','job_responsibilities', 'required_qualifications'):
            for keyseq, sequence in value.items():
                pure_item = []
                for item in sequence:
                    if(item == ''):
                        pure_item.append('N/A')
                    else:
                        # Remove the punctuation in the end of the text
                        # The meaning of the regular expression is start with 0 or more non alphanumeric or '(' or
                        # End with 0 or more non alphanumeric or ')'
                        item = re.sub("(^[^\w|(]*|[^\w|)]*$)","",item)
                        item = item.replace("\n"," ")
                        pure_item.append(item)
                value[keyseq] = pure_item
        elif value == ['']:
            listing_dict[key] = 'N/A'
        else:
            # Remove the punctuation in the end of the text
            # The meaning of the regular expression is start with 0 or more non alphanumeric or '(' or
            # End with 0 or more non alphanumeric or ')'
            value = re.sub("(^[^\w|(]*|[^\w|)]*$)","","".join(value))
            value = value.replace("\n"," ")
            value = "".join(value)
            listing_dict[key] = value
    # Store the result to the listing
    data['listing'].append(listing_dict)
print("There are",len(data['listing']),"job advertisements")

There are 500 job advertisements


In [15]:
job = data['listing']
job_dict = {}
for each_job in job:
    id_job = each_job['_id']
    qualification = []
    job_dict[id_job] = each_job['required_qualifications']

for key, value in job_dict.items():
    if(value!='N/A'):
        job_dict[key] = " ".join(value['qualification'])
    else:
        job_dict[key] = ""
print("Now there are",len(job_dict),"job advertisements")

Now there are 499 job advertisements


There are only 499 job advertisements left. This is because there are duplicate IDs in the job advertisements. We will keep going with 499 job advertisements.

## 2. Extract Resumes
In this part, we extract the informasi from the resumes by using the `nltk` library and regular expression.

In [16]:
import gensim
from nltk.tokenize import RegexpTokenizer

In [17]:
# tokenization function
def tokenizeRawData(fileid):
    """
        This function tokenizes a raw text document.
    """
    with open('resume/resume_('+str(fileid)+').txt','r') as infile:
        text = infile.read()
    tokenised_resume = tokenizer.tokenize(text) # tokenize each Reuters articles
    return (fileid, tokenised_resume)

In [18]:
# list of resumes
resume = [1, 2, 4, 7, 13, 17, 22, 23, 28, 30, 31, 36, 39, 44, 48, 55, 64, 66, 67, 72, 73, 79, 81, 89, 93, 94, 96, 97, 99, 101, 104, 105, 116, 123, 131, 133, 138, 145, 147, 149, 151, 154, 155, 164, 167, 169, 170, 173, 178, 182, 184, 188, 190, 199, 207, 211, 218, 224, 236, 251, 256, 263, 270, 273, 276, 284, 285, 292, 295, 296, 299, 303, 307, 308, 312, 315, 323, 324, 325, 329, 330, 336, 342, 346, 347, 352, 357, 360, 365, 371, 373, 378, 388, 398, 400, 402, 408, 410, 416, 418, 420, 421, 425, 428, 432, 435, 442, 444, 446, 447, 454, 466, 467, 472, 473, 474, 475, 476, 480, 481, 483, 487, 491, 494, 496, 497, 498, 504, 508, 510, 518, 522, 523, 525, 529, 533, 535, 537, 538, 539, 549, 557, 558, 566, 568, 571, 577, 579, 580, 581, 582, 587, 588, 589, 593, 594, 596, 599, 600, 604, 605, 607, 615, 618, 626, 632, 633, 636, 638, 641, 645, 648, 649, 654, 660, 662, 664, 670, 672, 675, 677, 683, 686, 693, 699, 703, 704, 705, 722, 724, 728, 732, 736, 739, 742, 745, 746, 751, 757, 771, 775, 780, 783, 784, 786, 792, 793, 797, 798, 800, 803, 804, 813, 815, 824, 825, 826, 830, 840, 844, 845, 847, 851, 854, 856, 857, 864]
# regular expression
tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?")

In [19]:
tokenized_result =  dict(tokenizeRawData(fileid) for fileid in resume)

## 3. Check The Similarity

In [20]:
#We will create a dictionary from the list of resumes. A dictionary maps every word to a number.
dictionary = gensim.corpora.Dictionary(tokenized_result.values())

In [21]:
#Now we will create a dictionary that contains a bag-of words which is the representation for a document just lists the number of times each word occurs in the document.
words_dict ={}
for key, value in tokenized_result.items():
    words_dict[key] = dictionary.doc2bow(value)

In [22]:
# Now we create a tf-idf model from the values of the dictionary
tf_idf = gensim.models.TfidfModel(words_dict.values())

# create a similarity measure object in tf-idf space.
sims = gensim.similarities.Similarity('output/',tf_idf[words_dict.values()], num_features=len(dictionary))

/Users/robert/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:718: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [23]:
# create a list of queries and convert them to tf-idf.
export_dict = {}
for key, value in job_dict.items():
    query_doc = tokenizer.tokenize(value)
    query_doc_bow = dictionary.doc2bow(query_doc)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    temp_dict = {}
    temp = (list(sims[query_doc_tf_idf]))
    for i in range(len(temp)):
        temp_dict[i] = temp[i]
    export_dict[key] = temp_dict
export_dict

{'20097': {0: 0.011466743,
  1: 0.010559787,
  2: 0.036756776,
  3: 0.013420165,
  4: 0.009155313,
  5: 0.0254859,
  6: 0.029231353,
  7: 0.022722796,
  8: 0.047693748,
  9: 0.028986074,
  10: 0.015181306,
  11: 0.03899995,
  12: 0.011889441,
  13: 0.05219411,
  14: 0.026211467,
  15: 0.021873273,
  16: 0.030354453,
  17: 0.044808075,
  18: 0.0068725427,
  19: 0.043448992,
  20: 0.018058183,
  21: 0.020199444,
  22: 0.02493723,
  23: 0.013815971,
  24: 0.02094074,
  25: 0.035664998,
  26: 0.012472906,
  27: 0.010193296,
  28: 0.040179327,
  29: 0.019592898,
  30: 0.027525738,
  31: 0.026562547,
  32: 0.052128717,
  33: 0.06374315,
  34: 0.014948708,
  35: 0.011112684,
  36: 0.010808919,
  37: 0.02321164,
  38: 0.010159178,
  39: 0.017297441,
  40: 0.0010514511,
  41: 0.021247152,
  42: 0.018351663,
  43: 0.012950213,
  44: 0.031734876,
  45: 0.022579286,
  46: 0.0074598836,
  47: 0.005146824,
  48: 0.0041804584,
  49: 0.004234221,
  50: 0.0077218716,
  51: 0.017389884,
  52: 0.00433721

In [24]:
# get the top 10 index by values
for fileid, value in export_dict.items():
    list_of_index = [key for key in sorted(value, key=value.get, reverse=True)[:10]]
    export_dict[fileid] = list_of_index
export_dict


{'20097': [179, 33, 91, 202, 221, 75, 84, 77, 168, 136],
 '20157': [58, 73, 183, 205, 127, 68, 179, 69, 30, 113],
 '20616': [213, 57, 105, 138, 216, 202, 33, 25, 2, 127],
 '20873': [179, 28, 117, 33, 106, 189, 104, 127, 202, 2],
 '20908': [28, 185, 13, 29, 168, 61, 179, 202, 58, 87],
 '21006': [136, 192, 117, 68, 224, 87, 22, 71, 180, 113],
 '21367': [10, 33, 106, 191, 6, 71, 104, 113, 216, 213],
 '21440': [106, 127, 28, 179, 119, 170, 123, 216, 113, 109],
 '21482': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 '21801': [213, 119, 113, 28, 127, 136, 44, 221, 117, 145],
 '22083': [137, 178, 4, 33, 217, 35, 117, 63, 225, 59],
 '22101': [179, 28, 118, 202, 162, 106, 185, 221, 87, 113],
 '22635': [127, 87, 213, 179, 110, 135, 184, 62, 29, 116],
 '22651': [179, 202, 113, 127, 42, 33, 129, 136, 209, 119],
 '22699': [179, 127, 124, 33, 102, 134, 195, 117, 42, 189],
 '22849': [185, 28, 179, 127, 13, 29, 169, 80, 170, 117],
 '22955': [179, 202, 127, 133, 104, 54, 155, 211, 107, 152],
 '23007': [28, 113, 33,

In [25]:
# get the resume number
for key, value in export_dict.items():
    list_of_resume = [resume[each] for each in value]
    export_dict[key] = list_of_resume
export_dict

{'20097': [675, 123, 378, 783, 847, 315, 347, 324, 638, 535],
 '20157': [236, 308, 693, 792, 504, 295, 675, 296, 104, 472],
 '20616': [815, 224, 435, 538, 826, 783, 123, 94, 4, 504],
 '20873': [675, 99, 476, 123, 442, 724, 432, 504, 783, 4],
 '20908': [99, 703, 44, 101, 638, 263, 675, 783, 236, 360],
 '21006': [535, 736, 476, 295, 856, 360, 81, 303, 677, 472],
 '21367': [31, 123, 442, 732, 22, 303, 432, 472, 826, 815],
 '21440': [442, 504, 99, 675, 481, 645, 494, 826, 472, 447],
 '21482': [1, 2, 4, 7, 13, 17, 22, 23, 28, 30],
 '21801': [815, 481, 472, 99, 504, 535, 167, 847, 476, 571],
 '22083': [537, 672, 13, 123, 830, 133, 476, 273, 857, 251],
 '22101': [675, 99, 480, 783, 615, 442, 703, 847, 360, 472],
 '22635': [504, 360, 815, 675, 454, 533, 699, 270, 101, 475],
 '22651': [675, 783, 472, 504, 155, 123, 510, 535, 800, 481],
 '22699': [675, 504, 496, 123, 425, 529, 745, 476, 155, 724],
 '22849': [703, 99, 675, 504, 44, 101, 641, 330, 645, 476],
 '22955': [675, 783, 504, 525, 432, 207

In [26]:
# export the result
bonus_vec= open("bonus_28243447.txt","w+")
for key, value in export_dict.items():
    bonus_vec.write(str(key)+":")
    bonus_vec.write(','.join(str(e) for e in value))
    bonus_vec.write("\n")
bonus_vec.close()

## 4. Assumption
* For job advertisement without required qualification, the top resumes will be based on the filename of the resumes.

## 5. Reference
https://www.oreilly.com/learning/how-do-i-compare-document-similarity-using-python